In [2]:
import shelve
import mkl_random as rnd
from tqdm import trange,tqdm
from numba import jit
import pandas as pd
pd.options.compute.use_numba = False
import itertools
import math

In [3]:
from mysetup import NotebookFinder
import sys

sys.meta_path.append(NotebookFinder())

from setup_work import *


importing Jupyter notebook from setup_work.ipynb


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
def gen_col_data(df):
    idx = df.index.to_flat_index().to_list()
    col_names = [f'col_{i}' for i in range(7)]
    df = pd.DataFrame(columns=col_names)
    for j,i_ in enumerate(idx):
        for k,c in zip(i_,col_names):
            df.loc[j,c] =k
    return df
orig_columns = ['binary__offer_expiration',
 'ordinal__income_range',
 'ordinal__no_visited_cold_drinks',
 'binary__travelled_more_than_15mins_for_offer',
 'ordinal__restaur_spend_less_than20',
 'nominal__marital_status',
 'nominal__restaurant_type',
 'ordinal__age',
 'binary__prefer_western_over_chinese',
 'binary__travelled_more_than_25mins_for_offer',
 'ordinal__no_visited_bars',
 'binary__gender',
 'nominal__car',
 'binary__restuarant_same_direction_house',
 'binary__cooks_regularly',
 'nominal__customer_type',
 'ordinal__qualif',
 'binary__is_foodie',
 'ordinal__no_take_aways',
 'nominal__job_industry',
 'binary__restuarant_opposite_direction_house',
 'binary__has_children',
 'ordinal__type_of_rest_rating',
 'interval__temperature',
 'ordinal__restaur_spend_greater_than20',
 'interval__travel_time',
 'interval__season',
 'ordinal__dest_distance',
 'binary__prefer_home_food',
 ]

In [5]:
data,eval_data = bring_data()

redundant,nr_data,nr_eval_data = elim_redund(data,eval_data)

Do you want to use new data? n


In [5]:
"""
    def gen_work_pile(self, work_size):
        work_pile = []
        n_work_pile = 0
        while(n_work_pile != work_size):
            pile = list(self.rng.choice(orig_columns,size=20,replace=False))
            pile_str = '|'.join(pile)            
            if pile_str not in self.pile_id:
                work_pile.append(pile)
                self.pile_id[pile_str]=0
                n_work_pile+=1
        return work_pile
        #     def gen_work_pile(self, work_size):
#         work_pile = list(itertools.combinations(self.binary,int(0.3*work_size))+ 
#         work_pile = [list(x) for x in work_pile]
#         return work_pile
#     def mark_permutations(self,pile_):
#         pile_len = len(pile_)
#         pile_perms = list(self.rng.choice(
#             list(itertools.permutations(pile_,pile_len)),
#             size=100,
#             replace=False
#         ))
#         for tmp_pile in pile_perms:
#             tmp_pile_str = '|'.join(tmp_pile)            
#             self.pile_id[tmp_pile_str]=0
"""
pass

In [45]:
list(itertools.permutations([1,2,3],3))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [12]:
import itertools

nominal = [x for x in orig_columns if "nominal_" in x]
ordinal = [x for x in orig_columns if "ordinal_" in x]
binary = [x for x in orig_columns if "binary_" in x]
interval = [x for x in orig_columns if "interval_" in x]

class FCombi(object):
    def __init__(self,size):
        self.search_space = orig_columns
        self.feature_coverage = []
        self.coverage_ratio = []
        self.max_coverage = np.inf
        self.nom_combi_min = [] 
        self.combi_data_min = []
        self.min_s = np.inf
        self.nom_combi_max = [] 
        self.combi_data_max = []
        self.max_s = np.NINF
        self.pile_id = {}
        self.size = size
        self.rng = np.random.default_rng()
        self.t_name = None
        self.max_similarity = np.NINF
        self.similar_features = []
        self.binary =  [x for x in self.search_space if "binary_" in x]
        self.nominal = [x for x in self.search_space if "nominal_" in x]
        self.ordinal =  [x for x in self.search_space if "ordinal_" in x]
        self.interval =  [x for x in self.search_space if "interval_" in x]
        
        
    def save(self):
        with shelve.open(DATA_OUTPUT+'combi_tracker') as tdb:
            next_idx = len([x for x in tdb if f"tracker_{tracker.size}" in x])+1
            tracker_name = f"tracker_{tracker.size}_{next_idx}"
            tdb[tracker_name] = tracker
            print(f'Saved Tracker under {tracker_name}')
        return


            
    def gen_work_pile(self, work_size):
        work_pile = []
        n_work_pile = 0
        while(n_work_pile != work_size):
            pile = list(self.rng.choice(orig_columns,size=12,replace=False))
            pile_str = '|'.join(pile)            
            if pile_str not in self.pile_id:
                work_pile.append(pile)
                self.pile_id[pile_str]=0
                n_work_pile+=1
#                 self.mark_permutations(pile)
        return work_pile
    
    def get_tname(self):
        with shelve.open(DATA_OUTPUT+'combi_tracker') as tdb:
            next_idx = len([x for x in tdb if f"tracker_{tracker.size}" in x])+1
            tracker_name = f"tracker_{tracker.size}_{next_idx}"
            self.t_name = tracker_name
        return self.t_name
    
    def save_instance(self):
        
        if self.t_name:
            with shelve.open(DATA_OUTPUT+'combi_tracker') as tdb:
                tdb[self.t_name] = tracker
                tdb[self.t_name+'__meta'] = tracker.__dict__
                print(f'Saved Tracker under {self.t_name}')
        else:
            tid = self.get_tname()
            with shelve.open(DATA_OUTPUT+'combi_tracker') as tdb:
                tdb[self.t_name] = tracker
                tdb[self.t_name+'__meta'] = tracker.__dict__
                print(f'Saved Tracker under {self.t_name}')            
        return

In [13]:
# nominal = [x for x in data.columns if "nominal_" in x]
# ordinal = [x for x in data.columns if "ordinal_" in x]
# binary = [x for x in data.columns if "binary_" in x]
# interval = [x for x in data.columns if "interval_" in x]

    
X = data.copy()
X_eval = eval_data.copy()
X['dummy'] = 1
X_eval['dummy']=1


def check_num_similar(pile):
    global X,X_eval
    acc = X.groupby(pile)['dummy'].sum()/X.groupby(pile)['dummy'].count()
    acc_eval = X_eval.groupby(pile)['dummy'].sum()/X_eval.groupby(pile)['dummy'].count()
    cmmn = len(acc.index.intersection(acc_eval.index))
    return cmmn

tracker = FCombi(122)

def process_pile(pile,df=data,df_eval=eval_data):
    global tracker
    pile_tc = df.groupby(pile)['target'].count()
    pile_tsum = df.groupby(pile)['target'].sum()
    res = pile_tsum/pile_tc
    #     check = any(item in pile for item in redundant)
    check=False
    for item in pile:
        if item in redundant:
            check=True
            break

    v,nu,sh,vc,redundancy    =res.var(), res.unique(), res.shape, res.value_counts(), check
    if vc.shape[0] <= tracker.min_s:
        tracker.min_s = vc.shape[0]
        tracker.nom_combi_min.append(pile)
        tracker.combi_data_min.append((v,nu,sh[0],vc))
    if vc.shape[0] >= tracker.max_s:
        tracker.max_s = vc.shape[0]
        tracker.nom_combi_max.append(pile)
        tracker.combi_data_max.append((v,nu,sh[0],vc))
    if pile_tc.value_counts()[1] <= tracker.max_coverage:
        tracker.max_coverage = pile_tc.value_counts()[1]
#         tracker.save_instance()        
        tracker.feature_coverage.append(pile)
        tracker.coverage_ratio.append(sh[0]/df.shape[0])
    if check_num_similar(pile) >= tracker.max_similarity:
        tracker.save_instance()        
        print(tracker.max_similarity)
        tracker.max_similarity = check_num_similar(pile)
        tracker.similar_features.append(pile)
        
    



In [14]:
print("Tracker_STats: ","Nom Combi Max: ",tracker.max_s,"Nom Combi Min: ",tracker.min_s,"  F Coverage: ",tracker.max_coverage," Max Similarity:",tracker.max_similarity)
# print("Similarity:    ","Nom Combi Max: ",check_num_similar(tracker.nom_combi_max[-1]),"Nom Combi Min: ",check_num_similar(tracker.nom_combi_min[-1]),"F Coverage: ",check_num_similar(tracker.feature_coverage[-1]))

Tracker_STats:  Nom Combi Max:  -inf Nom Combi Min:  inf   F Coverage:  inf  Max Similarity: -inf


In [ ]:
def rund(tracker=tracker,df=data):
    try:
        for i in trange(1_000_000):
            work_pile = tracker.gen_work_pile(3)
    #         with parallel_backend('threading',n_jobs=12):
    #             Parallel(require='sharedmem',n_jobs=12)(delayed(process_pile)(pile,df) for pile in work_pile[:500])
            for pile in work_pile:
                e=process_pile(pile,df=df)
    except KeyboardInterrupt as e:
        return
        
rund()
print("Tracker_STats: ","Nom Combi Max: ",tracker.max_s,"Nom Combi Min: ",tracker.min_s,"  F Coverage: ",tracker.max_coverage," Max Similarity:",tracker.max_similarity)
print("Similarity:    ","Nom Combi Max: ",check_num_similar(tracker.nom_combi_max[-1]),"Nom Combi Min: ",check_num_similar(tracker.nom_combi_min[-1]),"F Coverage: ",check_num_similar(tracker.feature_coverage[-1]))
# tracker.save_instance()

  0%|          | 0/1000000 [00:00<?, ?it/s]

Saved Tracker under tracker_122_1
-inf


  0%|          | 2/1000000 [00:00<82:33:16,  3.36it/s] 

Saved Tracker under tracker_122_1
905
Saved Tracker under tracker_122_1
1344


  0%|          | 7/1000000 [00:01<51:02:28,  5.44it/s]

...

  0%|          | 3941/1000000 [11:01<64:00:13,  4.32it/s]

Saved Tracker under tracker_122_1
2471


  1%|          | 10680/1000000 [29:52<87:19:52,  3.15it/s]

Saved Tracker under tracker_122_1
2504


  6%|▌         | 55256/1000000 [2:34:10<47:15:13,  5.55it/s]

In [15]:
import math
math.comb(29,12)

51895935

In [22]:
with shelve.open(DATA_OUTPUT+'combi_tracker') as tdb:
    tdb['tracker_44_2_meta'] = tracker.__dict__

In [16]:
X = data.copy()
X_eval = eval_data.copy()
X['dummy'] = 1
X_eval['dummy']=1
for fd in tqdm(tracker_7.nom_combi_min):
    acc = X.groupby(fd)['dummy'].sum()/X.groupby(fd)['dummy'].count()
    acc_eval = X_eval.groupby(fd)['dummy'].sum()/X_eval.groupby(fd)['dummy'].count()
    cmmn = len(acc.index.intersection(acc_eval.index))
#     if cmmn > 2000:
    print(cmmn)

 43%|████▎     | 6/14 [00:00<00:00, 29.26it/s]

1768
1951
1852
1928
1522
1564
851
684
652
983


100%|██████████| 14/14 [00:00<00:00, 28.61it/s]

629
572
599
773


In [26]:
len(binary)

11

In [47]:
X.groupby(tracker_7.nom_combi_min[-1])['dummy'].count()

nominal__spend_id  interval__temperature  nominal__job_industry  nominal__pref_profile  nominal__circumstance  nominal__restaurant_type  ordinal__income_range
0                  40                     0                      0                      12                     1                         9                        1
                                                                 15                     19                     2                         7                        1
                                          3                      4                      2                      0                         3                        1
                                                                 5                      8                      3                         3                        1
                                                                 7                      2                      1                         3                        1
                     

In [46]:
acc_idx = acc.index.to_flat_index().to_list()
acc_eval_idx = acc_eval.index.to_flat_index().to_list()

# gen_col_data(acc_eval_idx)

            

,col_0,col_1,col_2,col_3,col_4,col_5,col_6
0,0,40,0,8,2,2,9
1,0,40,0,14,1,1,3
2,0,40,1,0,8,2,3
3,0,40,1,12,2,4,3
4,0,40,3,5,20,1,3
...,...,...,...,...,...,...,...
5281,24,89,8,7,4,2,4
5282,24,89,8,11,1,0,4
5283,24,89,8,12,41,1,4
5284,24,89,15,13,41,2,4
